<center style="font-size: 26px"> <b>Проверка уровня знаний Python</b></center>

In [14]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import requests
from xml.etree import ElementTree
from datetime import datetime, timedelta
from collections import Counter

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv) 

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

#### Загрузите датасет

In [15]:
df = pd.read_csv('df2021-08-03.csv')

df

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01T13:02:48+0300,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09T08:13:01+0300,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21T15:24:16+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст..."
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T12:57:57+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31T17:10:49+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,Middle Software Developer C++/Python (relocati...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-16T13:45:47+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Полный день,Python;Git;Linux;iOS;C++;,Parallels,Москва,"Overview:Parallels Inc., a global leader in cr..."
1696,Python Tech Lead,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T14:40:13+0300,False,300000.0,NaN,RUR,От 1 года до 3 лет,Полный день,NaN,Финтех Айкью,Новосибирск,"Наша команда ищет тех. лида, задачами которого..."
1697,Senior Software Developer C++/Python (relocati...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-16T13:45:46+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Полный день,Python;Git;Linux;C++;x86;Windows;Unix;Virtuali...,Parallels,Москва,"Overview:Parallels Inc., a global leader in cr..."
1698,"Full-stack Developer (React, Python)",https://hh.ru/applicant/vacancy_response?vacan...,2021-08-02T16:36:09+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Git;PostgreSQL;Python;Blockchain;Crypto;React ...,Qenetex,Москва,"Мы – финтех компания, приглашаем вас принять у..."


### Практические задания:

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [16]:
# кэшируем курсы валют
exchange_rate_cache = {}

# функция для получения курса валюты с обработкой выходных и ошибок
def get_currency_rate(date, currency):
    if currency == 'RUR':
        return 1.0
    
    # если кэш уже содержит курс, возвращаем его
    cache_key = f"{date}_{currency}"
    if cache_key in exchange_rate_cache:
        return exchange_rate_cache[cache_key]
    
    # преобразуем дату в нужный формат
    try:
        date_obj = datetime.strptime(date, '%Y-%m-%d')
    except ValueError:
        return None
    
    # сдвигаем дату на предыдущий рабочий день, если это выходной
    while date_obj.weekday() >= 5:  # 5 - суббота, 6 - воскресенье
        date_obj -= timedelta(days=1)
    cbr_date = date_obj.strftime('%d/%m/%Y')
    
    # получаем курс валюты с сайта ЦБ РФ с обработкой ошибок
    url = f'http://www.cbr.ru/scripts/XML_daily.asp?date_req={cbr_date}'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        root = ElementTree.fromstring(response.content)  # парсим XML
    except requests.RequestException as e:
        print(f"Ошибка при запросе к API: {e}")
        return None
    
    for valute in root.findall('Valute'):
        char_code = valute.find('CharCode').text
        if char_code == currency:
            nominal = float(valute.find('Nominal').text.replace(',', '.'))
            value = float(valute.find('Value').text.replace(',', '.'))
            rate = value / nominal
            exchange_rate_cache[cache_key] = rate
            return rate
    return None

# преобразуем значения зарплат в рубли
def convert_salary(row):
    if pd.isna(row['currency']) or row['currency'] == 'RUR':
        return row['salary_from'], row['salary_to']
    
    rate = get_currency_rate(row['created'][:10], row['currency'])
    
    if pd.isna(rate) or rate is None:
        return row['salary_from'], row['salary_to']
    
    converted_from = row['salary_from'] * rate if not pd.isna(row['salary_from']) else None
    converted_to = row['salary_to'] * rate if not pd.isna(row['salary_to']) else None
    return converted_from, converted_to

# применяем функцию к DataFrame
df[['converted_salary_from', 'converted_salary_to']] = df.apply(convert_salary, axis=1, result_type='expand')

# сохраняем оригинальные значения, если конвертация не удалась
df['salary_from'] = df['converted_salary_from'].fillna(df['salary_from'])
df['salary_to'] = df['converted_salary_to'].fillna(df['salary_to'])

df.drop(['converted_salary_from', 'converted_salary_to'], axis=1, inplace=True)

#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [17]:
# фильтруем вакансии по опыту и тестированию
filtered_df = df[
    (df['experience'] == 'От 3 до 6 лет') & 
    (df['has_test'] == False)
]

# рассчитываем максимальную зарплату
filtered_df['max_salary'] = filtered_df[['salary_from', 'salary_to']].max(axis=1)

# ищем вакансию с максимальной зарплатой
max_salary_row = filtered_df.loc[filtered_df['max_salary'].idxmax()]

print(f"Должность: {max_salary_row['vacancy']}")
print(f"Зарплата: {max_salary_row['max_salary']:,.0f} ₽")
print(f"Компания: {max_salary_row['employer']}")
print(f"Требуемый опыт: {max_salary_row['experience']}")
print(f"Ссылка: {max_salary_row['url']}")

Должность: Full-stack Разработчик React, Python
Зарплата: 500,000 ₽
Компания: MINT NFT
Требуемый опыт: От 3 до 6 лет
Ссылка: https://hh.ru/applicant/vacancy_response?vacancyId=46458400


#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [18]:
# сбор данных о навыках и очистка от пробелов
all_skills = df['skills'].dropna().str.lower().str.split(';').sum()
all_skills = [skill.strip() for skill in all_skills if skill.strip()]

# подсчет количества вхождений каждого навыка
skill_counts = Counter(all_skills)

top_30_skills = skill_counts.most_common(30)

top_skills_df = pd.DataFrame(top_30_skills, columns=['Навык', 'Количество вакансий'])

top_skills_df

,Навык,Количество вакансий
0,python,1442
1,git,586
2,sql,501
3,postgresql,499
4,django framework,467
5,linux,463
6,docker,180
7,flask,177
8,javascript,149
9,английский язык,141


#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [19]:
# фильтруем вакансии, расширяя регулярное выражение
qa_mask = (
    (
        df['vacancy'].str.lower().str.contains(
            r'qa|quality assurance|качество|тестир|тест|тестировщик|qa engineer|software tester|automation qa',
            regex=True, na=False
        )
    ) 
    &
    ~df['vacancy'].str.lower().str.contains(r'преподаватель|учитель', regex=True, na=False)
)
qa_vacancies = df[qa_mask].copy()

# исключаем строки, где валюта не RUR (конвертация не удалась)
qa_vacancies = qa_vacancies[qa_vacancies['currency'].isna() | (qa_vacancies['currency'] == 'RUR')]

# рассчитываем среднюю зарплату только для строк, где есть хотя бы одна граница зарплаты
qa_vacancies = qa_vacancies.dropna(subset=['salary_from', 'salary_to'], how='all')
qa_vacancies['avg_salary'] = qa_vacancies[['salary_from', 'salary_to']].mean(axis=1, skipna=True)

# группируем и сортируем 
top_cities = (
    qa_vacancies
    .groupby('area', observed=True)['avg_salary']
    .mean()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    .rename(columns={'area': 'Город', 'avg_salary': 'Средняя ЗП'})
)

# форматируем зарплату
top_cities['Средняя ЗП'] = top_cities['Средняя ЗП'].apply(lambda x: f"{x:,.0f} ₽".replace(',', ' '))
top_cities

,Город,Средняя ЗП
0,Тамбов,200 000 ₽
1,Москва,186 500 ₽
2,Санкт-Петербург,172 273 ₽


In [20]:
display(qa_vacancies)

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description,avg_salary
52,QA Engineer (Python) / тестировщик,https://hh.ru/applicant/vacancy_response?vacan...,2021-08-02T15:49:10+0300,False,70000.0,180000.0,RUR,От 1 года до 3 лет,Полный день,Atlassian Jira;Функциональное тестирование;Tes...,BelkaCar,Москва,"BelkaCar — каршеринг, работающий на рынках Мос...",125000.0
121,QA engineer (Data + Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-29T17:11:40+0300,False,250000.0,NaN,RUR,От 3 до 6 лет,Удаленная работа,Python;SQL;Английский язык;Kafka;ETL;DWH;Data;,Exness,Москва,"Exness, a leading foreign exchange broker in t...",250000.0
211,Senior QA Automation Engineer (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31T15:08:47+0300,False,300000.0,NaN,RUR,От 3 до 6 лет,Удаленная работа,Автоматизация тестирования;Функциональное тест...,Scalable Solutions,Москва,Scalable Solutions – международная технологиче...,300000.0
275,Middle Python QA Engineer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T09:51:12+0300,False,100000.0,100000.0,RUR,От 1 года до 3 лет,Полный день,Python;SQL;,Национальный исследовательский университет Выс...,Москва,"Мы, Высшая школа экономики – один из крупнейши...",100000.0
309,Инженер по автоматизированному тестированию (Q...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-22T10:08:41+0300,False,90000.0,140000.0,RUR,От 1 года до 3 лет,Полный день,С;Python;Linux;ООП;Networking;Black box testin...,АМИКОН,Москва,"Компании «АМИКОН» уже более 25 лет, и она явля...",115000.0
395,QA Engineer(python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-02T10:50:26+0300,False,NaN,175000.0,RUR,От 1 года до 3 лет,Удаленная работа,Python;Postman;Английский язык;Selenium IDE;,Wanted,Санкт-Петербург,Вакансия: : QA Engineer(python) Адрес: Санкт-П...,175000.0
458,QA Automation Engineer Python (mobile),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-19T11:00:44+0300,False,150000.0,250000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;Atlassian Jira;SQL;Linux;,KINDDA,Санкт-Петербург,Обязанности: Анализ требований; Настройка ...,200000.0
502,QA Automation Engineer / Тестировщик (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-27T10:50:03+0300,False,100000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;Git;SQL;Atlassian Jira;Docker;,ТаймВэб,Санкт-Петербург,Timeweb — экосистема международных технологиче...,100000.0
525,Senior QA Automation Engineer (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01T15:08:35+0300,False,300000.0,NaN,RUR,От 3 до 6 лет,Удаленная работа,Автоматизация тестирования;Функциональное тест...,Scalable Solutions,Санкт-Петербург,Scalable Solutions – международная технологиче...,300000.0
553,Automation QA (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-29T22:09:37+0300,False,NaN,250000.0,RUR,Нет опыта,Полный день,Python;Git;Linux;Docker;Jenkins;QA;QA Automati...,Global Hunter,Москва,"Крупная российская IT компания, разрабатывающа...",250000.0


#### 5. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий. 

In [21]:
def search_vacancies(df, min_salary, max_salary, has_test, title_text, skill):

    filtered = df[(df["salary_from"] >= min_salary) & (df["salary_to"] <= max_salary)]

    filtered = filtered[filtered["has_test"] == has_test]
    
    filtered = filtered[filtered["vacancy"].str.contains(title_text, case=False, na=False)]
    
    filtered = filtered[filtered["skills"].str.contains(skill, case=False, na=False)]
    
    avg_salary = filtered[["salary_from", "salary_to"]].mean(axis=1).mean()
    
    return {
        "count": len(filtered),
        "avg_salary": round(avg_salary, 2) if not pd.isna(avg_salary) else 0,
        "links": filtered["url"].tolist()}

In [22]:
# пример
search_vacancies(df, min_salary=100000, max_salary=300000, has_test=False, title_text="QA", skill="Python")

{'count': 6,
 'avg_salary': np.float64(175833.33),
 'links': ['https://hh.ru/applicant/vacancy_response?vacancyId=46582713',
  'https://hh.ru/applicant/vacancy_response?vacancyId=43954953',
  'https://hh.ru/applicant/vacancy_response?vacancyId=45062852',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46339361',
  'https://hh.ru/applicant/vacancy_response?vacancyId=43394113',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46212451']}

#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.

In [23]:
def search_vacancies(
    df,
    min_salary: float = None,
    max_salary: float = None,
    has_test: bool = None,
    title_text: str = None,
    skill: str = None
) -> dict:

    filtered2 = df.copy()
    
    if min_salary is not None: filtered2 = filtered2[(filtered2["salary_from"] >= min_salary) | (filtered2["salary_to"] >= min_salary)]
    if max_salary is not None: filtered2 = filtered2[(filtered2["salary_from"] <= max_salary) | (filtered2["salary_to"] <= max_salary)]
    
    if has_test is not None: filtered2 = filtered2[filtered2["has_test"] == has_test]
    
    if title_text: filtered2 = filtered2[filtered2["vacancy"].str.contains(title_text, case=False, na=False)]
    
    if skill: filtered2 = filtered2[filtered2["skills"].str.contains(skill, case=False, na=False)]
    
    avg_salary = filtered2[["salary_from", "salary_to"]].mean(axis=1).mean()
    
    return {"count": len(filtered2), 
            "avg_salary": round(avg_salary, 2) if not pd.isna(avg_salary) else 0, 
            "links": filtered2["url"].tolist()}

In [24]:
# пример
search_vacancies(df, min_salary=300000, title_text="senior", skill="sql")

{'count': 13,
 'avg_salary': np.float64(292174.7),
 'links': ['https://hh.ru/applicant/vacancy_response?vacancyId=46670558',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46670557',
  'https://hh.ru/applicant/vacancy_response?vacancyId=42615019',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46497746',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46496928',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46563619',
  'https://hh.ru/applicant/vacancy_response?vacancyId=39339342',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46700993',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46124734',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46212591',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46705713',
  'https://hh.ru/applicant/vacancy_response?vacancyId=45355018',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46417735']}